In [1]:
%matplotlib inline
import argparse

from ems.algorithms.analysis.analyze_percent_coverage import AnalyzePercentCoverage
from ems.algorithms.selection.dispatch_fastest_ambulance import BestTravelTimeAlgorithm
from ems.data.filters import kmeans_select_bases
from ems.data.tijuana import CSVTijuanaDataset
from ems.models.ambulance import Ambulance
from ems.settings import Settings
from ems.simulators.dispatcher_simulator import DispatcherSimulator

def go(settings:str="small_hans", ambulances:int = 7, bases:int = 7):

    clargs = argparse.Namespace(settings=settings, ambulances = ambulances, bases = bases)

    # Initialize settings
    settings = Settings(debug=True,
                        args=clargs
                       )

    # Initialize dataset
    dataset = CSVTijuanaDataset(demands_file_path=settings.demands_file,
                                bases_file_path=settings.bases_file,
                                cases_file_path=settings.cases_file,
                                travel_times_file_path=settings.travel_times_file)

    # Initialize ambulance_selection
    ambulance_select = BestTravelTimeAlgorithm(travel_times=dataset.travel_times)

    # Initialize demand_coverage
    determine_coverage = []
    determine_coverage.append(AnalyzePercentCoverage(travel_times=dataset.travel_times))

    # Select bases
    chosen_base_locations = kmeans_select_bases(dataset.bases, dataset.travel_times)

    # Generate ambulances - random base placement (may want to abstract into function)
    ambulances = []
    for index in range(settings.num_ambulances):
        ambulance = Ambulance(id=index,
                              base=chosen_base_locations[index],
                              location=chosen_base_locations[index])
        ambulances.append(ambulance)

    # Initialize the simulator
    sim = DispatcherSimulator(ambulances=ambulances,
                              cases=dataset.cases,
                              ambulance_selector=ambulance_select,
                              coverage_alg=determine_coverage,
                              plot=settings.plot
                              )

    # Start the whole thing
    finished_cases = sim.run()



In [2]:
configurations = dict()

lots_of_parameters = [(number_of_ambulances, number_of_bases) \
                      for number_of_ambulances in range(10, 2, -6) \
                      for number_of_bases in range(10, 2, -6)]

print('Parameters: ', lots_of_parameters)
print()
print('Length: ', len(lots_of_parameters))

parameters = [num for num in range (16, 0, -5)]
parameters

Parameters:  [(10, 10), (10, 4), (4, 10), (4, 4)]

Length:  4


[16, 11, 6, 1]

In [3]:
from multiprocessing import Process as Proc
from time import sleep
sleeptime = 10

processes = []
joining = []

# for params in lots_of_parameters:
#     p = Proc(target=go, 
#              kwargs={"settings": "small_hans" ,
#                      "ambulances" : params[0], 
#                      "bases" : params[1]}, 
#              name='ambs ' + str(params[0]) + ' bases ' + str(params[1])
#             )
#     processes.append(p)


for params in parameters:
    p = Proc(target=go, 
             kwargs={"settings": "hans_med" ,
                     "ambulances" : params, 
                     "bases" : params}, 
             name='ambs ' + str(params) + ' bases ' + str(params)
            )
    processes.append(p)

In [4]:
proc = processes.pop()
print(proc.name)
proc.start()
joining.append(proc)

ambs 1 bases 1
Default init_bases(): Kmeans init bases

Current Time: 2014-01-01 00:13:00
Current Event: Start Case
Coverage: 28.000000000000004 %. 
Starting case 138 which was recorded at 2014-01-01 00:13:00
Ambulance 0 chosen with one-way travel time 0:13:42 (total duration: 0 days 00:47:24)
Deploying ambulance 0 at time 2014-01-01 00:13:00
Delay on this case: 0 days 00:00:00
Busy ambulances:  [0]
Pending cases:  []

Current Time: 2014-01-01 00:21:00
Current Event: Delay Case
Coverage: 0.0 %. 
No available ambulances to deal with incoming case: 36 at 2014-01-01 00:21:00
Busy ambulances:  [0]
Pending cases:  [36]

Current Time: 2014-01-01 00:46:00
Current Event: Delay Case
Coverage: 0.0 %. 
No available ambulances to deal with incoming case: 141 at 2014-01-01 00:46:00
Busy ambulances:  [0]
Pending cases:  [36, 141]

Current Time: 2014-01-01 00:59:00
Current Event: Delay Case
Coverage: 0.0 %. 
No available ambulances to deal with incoming case: 109 at 2014-01-01 00:59:00
Busy ambulance

Deploying ambulance 0 at time 2014-01-01 05:16:12
Delay on this case: 0 days 02:00:12
Busy ambulances:  [0]
Pending cases:  []

Current Time: 2014-01-01 05:55:20
Current Event: Retire Ambulance
Coverage: 0.0 %. 
Retiring ambulance 0 at time 2014-01-01 05:55:20
Busy ambulances:  []
Pending cases:  []

Current Time: 2014-01-01 06:43:00
Current Event: Start Case
Coverage: 28.000000000000004 %. 
Starting case 97 which was recorded at 2014-01-01 06:43:00
Ambulance 0 chosen with one-way travel time 0:09:00 (total duration: 0 days 00:38:00)
Deploying ambulance 0 at time 2014-01-01 06:43:00
Delay on this case: 0 days 00:00:00
Busy ambulances:  [0]
Pending cases:  []

Current Time: 2014-01-01 06:51:00
Current Event: Delay Case
Coverage: 0.0 %. 
No available ambulances to deal with incoming case: 5 at 2014-01-01 06:51:00
Busy ambulances:  [0]
Pending cases:  [5]

Current Time: 2014-01-01 07:10:00
Current Event: Delay Case
Coverage: 0.0 %. 
No available ambulances to deal with incoming case: 37 a

Current Event: Delay Case
Coverage: 0.0 %. 
No available ambulances to deal with incoming case: 116 at 2014-01-01 10:28:00
Busy ambulances:  [0]
Pending cases:  [29, 113, 22, 96, 85, 116]

Current Time: 2014-01-01 10:30:00
Current Event: Delay Case
Coverage: 0.0 %. 
No available ambulances to deal with incoming case: 7 at 2014-01-01 10:30:00
Busy ambulances:  [0]
Pending cases:  [29, 113, 22, 96, 85, 116, 7]

Current Time: 2014-01-01 10:47:54
Current Event: Retire Ambulance
Coverage: 0.0 %. 
Retiring ambulance 0 at time 2014-01-01 10:47:54
Busy ambulances:  []
Pending cases:  [29, 113, 22, 96, 85, 116, 7]

Current Time: 2014-01-01 10:47:54
Current Event: Start Case
Coverage: 28.000000000000004 %. 
Starting case 29 which was recorded at 2014-01-01 09:14:00
Ambulance 0 chosen with one-way travel time 0:09:00 (total duration: 0 days 00:38:00)
Deploying ambulance 0 at time 2014-01-01 10:47:54
Delay on this case: 0 days 01:33:54
Busy ambulances:  [0]
Pending cases:  [113, 22, 96, 85, 116, 7


Current Time: 2014-01-01 14:53:52
Current Event: Start Case
Coverage: 28.000000000000004 %. 
Starting case 7 which was recorded at 2014-01-01 10:30:00
Ambulance 0 chosen with one-way travel time 0:09:00 (total duration: 0 days 00:38:00)
Deploying ambulance 0 at time 2014-01-01 14:53:52
Delay on this case: 0 days 04:23:52
Busy ambulances:  [0]
Pending cases:  [32, 114, 8, 115, 146, 118, 9]

Current Time: 2014-01-01 15:15:00
Current Event: Delay Case
Coverage: 0.0 %. 
No available ambulances to deal with incoming case: 95 at 2014-01-01 15:15:00
Busy ambulances:  [0]
Pending cases:  [32, 114, 8, 115, 146, 118, 9, 95]

Current Time: 2014-01-01 15:31:52
Current Event: Retire Ambulance
Coverage: 0.0 %. 
Retiring ambulance 0 at time 2014-01-01 15:31:52
Busy ambulances:  []
Pending cases:  [32, 114, 8, 115, 146, 118, 9, 95]

Current Time: 2014-01-01 15:31:52
Current Event: Start Case
Coverage: 28.000000000000004 %. 
Starting case 32 which was recorded at 2014-01-01 11:30:00
Ambulance 0 chosen

Busy ambulances:  [0]
Pending cases:  [118, 9, 95, 10, 11, 144, 117, 12, 145, 119, 23, 13]

Current Time: 2014-01-01 18:37:00
Current Event: Delay Case
Coverage: 0.0 %. 
No available ambulances to deal with incoming case: 142 at 2014-01-01 18:37:00
Busy ambulances:  [0]
Pending cases:  [118, 9, 95, 10, 11, 144, 117, 12, 145, 119, 23, 13, 142]

Current Time: 2014-01-01 18:41:00
Current Event: Delay Case
Coverage: 0.0 %. 
No available ambulances to deal with incoming case: 147 at 2014-01-01 18:41:00
Busy ambulances:  [0]
Pending cases:  [118, 9, 95, 10, 11, 144, 117, 12, 145, 119, 23, 13, 142, 147]

Current Time: 2014-01-01 18:41:52
Current Event: Retire Ambulance
Coverage: 0.0 %. 
Retiring ambulance 0 at time 2014-01-01 18:41:52
Busy ambulances:  []
Pending cases:  [118, 9, 95, 10, 11, 144, 117, 12, 145, 119, 23, 13, 142, 147]

Current Time: 2014-01-01 18:41:52
Current Event: Start Case
Coverage: 28.000000000000004 %. 
Starting case 118 which was recorded at 2014-01-01 13:41:00
Ambulanc

...


Current Time: 2014-01-07 18:15:00
Current Event: Delay Case
Coverage: 0.0 %. 
No available ambulances to deal with incoming case: 515 at 2014-01-07 18:15:00
Busy ambulances:  [0]
Pending cases:  [257, 367, 263, 250, 295, 260, 261, 334, 243, 264, 268, 231, 368, 252, 237, 270, 369, 269, 271, 276, 294, 293, 335, 262, 272, 244, 371, 253, 218, 290, 299, 278, 251, 372, 233, 279, 266, 337, 245, 249, 280, 375, 376, 521, 303, 522, 354, 321, 523, 329, 309, 310, 306, 524, 374, 516, 305, 355, 311, 380, 525, 314, 526, 326, 307, 519, 356, 342, 527, 320, 528, 357, 308, 343, 304, 344, 379, 535, 517, 313, 323, 345, 529, 351, 378, 327, 353, 346, 359, 530, 316, 347, 338, 373, 520, 531, 340, 322, 360, 317, 532, 348, 341, 325, 318, 319, 324, 361, 533, 349, 362, 386, 449, 450, 412, 383, 423, 394, 413, 432, 382, 388, 401, 399, 451, 536, 442, 433, 414, 425, 537, 534, 387, 434, 443, 385, 539, 384, 444, 389, 538, 415, 426, 454, 395, 390, 406, 416, 427, 541, 469, 436, 417, 542, 428, 408, 392, 418, 437, 352, 445


Current Time: 2014-01-07 18:31:00
Current Event: Delay Case
Coverage: 0.0 %. 
No available ambulances to deal with incoming case: 486 at 2014-01-07 18:31:00
Busy ambulances:  [0]
Pending cases:  [367, 263, 250, 295, 260, 261, 334, 243, 264, 268, 231, 368, 252, 237, 270, 369, 269, 271, 276, 294, 293, 335, 262, 272, 244, 371, 253, 218, 290, 299, 278, 251, 372, 233, 279, 266, 337, 245, 249, 280, 375, 376, 521, 303, 522, 354, 321, 523, 329, 309, 310, 306, 524, 374, 516, 305, 355, 311, 380, 525, 314, 526, 326, 307, 519, 356, 342, 527, 320, 528, 357, 308, 343, 304, 344, 379, 535, 517, 313, 323, 345, 529, 351, 378, 327, 353, 346, 359, 530, 316, 347, 338, 373, 520, 531, 340, 322, 360, 317, 532, 348, 341, 325, 318, 319, 324, 361, 533, 349, 362, 386, 449, 450, 412, 383, 423, 394, 413, 432, 382, 388, 401, 399, 451, 536, 442, 433, 414, 425, 537, 534, 387, 434, 443, 385, 539, 384, 444, 389, 538, 415, 426, 454, 395, 390, 406, 416, 427, 541, 469, 436, 417, 542, 428, 408, 392, 418, 437, 352, 445, 407


Current Time: 2014-01-07 19:36:00
Current Event: Delay Case
Coverage: 0.0 %. 
No available ambulances to deal with incoming case: 507 at 2014-01-07 19:36:00
Busy ambulances:  [0]
Pending cases:  [263, 250, 295, 260, 261, 334, 243, 264, 268, 231, 368, 252, 237, 270, 369, 269, 271, 276, 294, 293, 335, 262, 272, 244, 371, 253, 218, 290, 299, 278, 251, 372, 233, 279, 266, 337, 245, 249, 280, 375, 376, 521, 303, 522, 354, 321, 523, 329, 309, 310, 306, 524, 374, 516, 305, 355, 311, 380, 525, 314, 526, 326, 307, 519, 356, 342, 527, 320, 528, 357, 308, 343, 304, 344, 379, 535, 517, 313, 323, 345, 529, 351, 378, 327, 353, 346, 359, 530, 316, 347, 338, 373, 520, 531, 340, 322, 360, 317, 532, 348, 341, 325, 318, 319, 324, 361, 533, 349, 362, 386, 449, 450, 412, 383, 423, 394, 413, 432, 382, 388, 401, 399, 451, 536, 442, 433, 414, 425, 537, 534, 387, 434, 443, 385, 539, 384, 444, 389, 538, 415, 426, 454, 395, 390, 406, 416, 427, 541, 469, 436, 417, 542, 428, 408, 392, 418, 437, 352, 445, 407, 429


Current Time: 2014-01-07 20:47:00
Current Event: Delay Case
Coverage: 0.0 %. 
No available ambulances to deal with incoming case: 492 at 2014-01-07 20:47:00
Busy ambulances:  [0]
Pending cases:  [295, 260, 261, 334, 243, 264, 268, 231, 368, 252, 237, 270, 369, 269, 271, 276, 294, 293, 335, 262, 272, 244, 371, 253, 218, 290, 299, 278, 251, 372, 233, 279, 266, 337, 245, 249, 280, 375, 376, 521, 303, 522, 354, 321, 523, 329, 309, 310, 306, 524, 374, 516, 305, 355, 311, 380, 525, 314, 526, 326, 307, 519, 356, 342, 527, 320, 528, 357, 308, 343, 304, 344, 379, 535, 517, 313, 323, 345, 529, 351, 378, 327, 353, 346, 359, 530, 316, 347, 338, 373, 520, 531, 340, 322, 360, 317, 532, 348, 341, 325, 318, 319, 324, 361, 533, 349, 362, 386, 449, 450, 412, 383, 423, 394, 413, 432, 382, 388, 401, 399, 451, 536, 442, 433, 414, 425, 537, 534, 387, 434, 443, 385, 539, 384, 444, 389, 538, 415, 426, 454, 395, 390, 406, 416, 427, 541, 469, 436, 417, 542, 428, 408, 392, 418, 437, 352, 445, 407, 429, 438, 419


Current Time: 2014-01-07 21:56:20
Current Event: Start Case


In [5]:
sleep(sleeptime)
proc = processes.pop()
print(proc.name)
proc.start()
joining.append(proc)

ambs 6 bases 6
Default init_bases(): Kmeans init bases

Current Time: 2014-01-01 00:13:00
Current Event: Start Case
Coverage: 85.0 %. 
Starting case 138 which was recorded at 2014-01-01 00:13:00
Ambulance 5 chosen with one-way travel time 0:09:42 (total duration: 0 days 00:39:24)
Deploying ambulance 5 at time 2014-01-01 00:13:00
Delay on this case: 0 days 00:00:00
Busy ambulances:  [5]
Pending cases:  []

Current Time: 2014-01-01 00:21:00
Current Event: Start Case
Coverage: 81.0 %. 
Starting case 36 which was recorded at 2014-01-01 00:21:00
Ambulance 0 chosen with one-way travel time 0:09:00 (total duration: 0 days 00:38:00)
Deploying ambulance 0 at time 2014-01-01 00:21:00
Delay on this case: 0 days 00:00:00
Busy ambulances:  [0, 5]
Pending cases:  []

Current Time: 2014-01-01 00:46:00
Current Event: Start Case
Coverage: 62.0 %. 
Starting case 141 which was recorded at 2014-01-01 00:46:00
Ambulance 1 chosen with one-way travel time 0:07:24 (total duration: 0 days 00:34:48)
Deploying a


Current Time: 2014-01-01 08:01:00
Current Event: Start Case
Coverage: 69.0 %. 
Starting case 26 which was recorded at 2014-01-01 08:01:00
Ambulance 0 chosen with one-way travel time 0:09:00 (total duration: 0 days 00:38:00)
Deploying ambulance 0 at time 2014-01-01 08:01:00
Delay on this case: 0 days 00:00:00
Busy ambulances:  [0, 1]
Pending cases:  []

Current Time: 2014-01-01 08:06:10
Current Event: Retire Ambulance
Coverage: 44.0 %. 
Retiring ambulance 1 at time 2014-01-01 08:06:10
Busy ambulances:  [0]
Pending cases:  []

Current Time: 2014-01-01 08:33:00
Current Event: Start Case
Coverage: 66.0 %. 
Starting case 112 which was recorded at 2014-01-01 08:33:00
Ambulance 2 chosen with one-way travel time 0:07:29 (total duration: 0 days 00:34:58)
Deploying ambulance 2 at time 2014-01-01 08:33:00
Delay on this case: 0 days 00:00:00
Busy ambulances:  [0, 2]
Pending cases:  []

Current Time: 2014-01-01 08:36:00
Current Event: Start Case
Coverage: 50.0 %. 
Starting case 6 which was recorde


Current Time: 2014-01-01 11:35:18
Current Event: Retire Ambulance
Coverage: 59.0 %. 
Retiring ambulance 4 at time 2014-01-01 11:35:18
Busy ambulances:  [0]
Pending cases:  []

Current Time: 2014-01-01 12:08:00
Current Event: Retire Ambulance
Coverage: 66.0 %. 
Retiring ambulance 0 at time 2014-01-01 12:08:00
Busy ambulances:  []
Pending cases:  []

Current Time: 2014-01-01 12:20:00
Current Event: Start Case
Coverage: 85.0 %. 
Starting case 114 which was recorded at 2014-01-01 12:20:00
Ambulance 0 chosen with one-way travel time 0:09:00 (total duration: 0 days 00:38:00)
Deploying ambulance 0 at time 2014-01-01 12:20:00
Delay on this case: 0 days 00:00:00
Busy ambulances:  [0]
Pending cases:  []

Current Time: 2014-01-01 12:53:00
Current Event: Start Case
Coverage: 66.0 %. 
Starting case 8 which was recorded at 2014-01-01 12:53:00
Ambulance 3 chosen with one-way travel time 0:10:58 (total duration: 0 days 00:41:56)
Deploying ambulance 3 at time 2014-01-01 12:53:00
Delay on this case: 0 

Busy ambulances:  []
Pending cases:  []

Current Time: 2014-01-01 17:12:00
Current Event: Start Case
Coverage: 85.0 %. 
Starting case 117 which was recorded at 2014-01-01 17:12:00
Ambulance 0 chosen with one-way travel time 0:09:00 (total duration: 0 days 00:38:00)
Deploying ambulance 0 at time 2014-01-01 17:12:00
Delay on this case: 0 days 00:00:00
Busy ambulances:  [0]
Pending cases:  []

Current Time: 2014-01-01 17:24:00
Current Event: Start Case
Coverage: 66.0 %. 
Starting case 12 which was recorded at 2014-01-01 17:24:00
Ambulance 3 chosen with one-way travel time 0:10:58 (total duration: 0 days 00:41:56)
Deploying ambulance 3 at time 2014-01-01 17:24:00
Delay on this case: 0 days 00:00:00
Busy ambulances:  [0, 3]
Pending cases:  []

Current Time: 2014-01-01 17:40:00
Current Event: Start Case
Coverage: 54.0 %. 
Starting case 145 which was recorded at 2014-01-01 17:40:00
Ambulance 1 chosen with one-way travel time 0:18:05 (total duration: 0 days 00:56:10)
Deploying ambulance 1 at t


Current Time: 2014-01-01 20:17:00
Current Event: Retire Ambulance
Coverage: 25.0 %. 
Retiring ambulance 0 at time 2014-01-01 20:17:00
Busy ambulances:  [1, 3, 4]
Pending cases:  []

Current Time: 2014-01-01 20:19:00
Current Event: Start Case
Coverage: 50.0 %. 
Starting case 120 which was recorded at 2014-01-01 20:19:00
Ambulance 0 chosen with one-way travel time 0:09:00 (total duration: 0 days 00:38:00)
Deploying ambulance 0 at time 2014-01-01 20:19:00
Delay on this case: 0 days 00:00:00
Busy ambulances:  [0, 1, 3, 4]
Pending cases:  []

Current Time: 2014-01-01 20:39:08
Current Event: Retire Ambulance
Coverage: 25.0 %. 
Retiring ambulance 3 at time 2014-01-01 20:39:08
Busy ambulances:  [0, 1, 4]
Pending cases:  []

Current Time: 2014-01-01 20:44:50
Current Event: Retire Ambulance
Coverage: 37.0 %. 
Retiring ambulance 1 at time 2014-01-01 20:44:50
Busy ambulances:  [0, 4]
Pending cases:  []

Current Time: 2014-01-01 20:57:00
Current Event: Retire Ambulance
Coverage: 59.0 %. 
Retiring 

In [6]:
sleep(sleeptime)
proc = processes.pop()
print(proc.name)
proc.start()
joining.append(proc)

ambs 11 bases 11
Default init_bases(): Kmeans init bases

Current Time: 2014-01-01 00:13:00
Current Event: Start Case
Coverage: 95.0 %. 
Starting case 138 which was recorded at 2014-01-01 00:13:00
Ambulance 5 chosen with one-way travel time 0:09:42 (total duration: 0 days 00:39:24)
Deploying ambulance 5 at time 2014-01-01 00:13:00
Delay on this case: 0 days 00:00:00
Busy ambulances:  [5]
Pending cases:  []

Current Time: 2014-01-01 00:21:00
Current Event: Start Case
Coverage: 91.0 %. 
Starting case 36 which was recorded at 2014-01-01 00:21:00
Ambulance 0 chosen with one-way travel time 0:09:00 (total duration: 0 days 00:38:00)
Deploying ambulance 0 at time 2014-01-01 00:21:00
Delay on this case: 0 days 00:00:00
Busy ambulances:  [0, 5]
Pending cases:  []

Current Time: 2014-01-01 00:46:00
Current Event: Start Case
Coverage: 75.0 %. 
Starting case 141 which was recorded at 2014-01-01 00:46:00
Ambulance 1 chosen with one-way travel time 0:07:24 (total duration: 0 days 00:34:48)
Deploying


Current Time: 2014-01-01 07:57:22
Current Event: Retire Ambulance
Coverage: 95.0 %. 
Retiring ambulance 8 at time 2014-01-01 07:57:22
Busy ambulances:  []
Pending cases:  []

Current Time: 2014-01-01 08:01:00
Current Event: Start Case
Coverage: 95.0 %. 
Starting case 26 which was recorded at 2014-01-01 08:01:00
Ambulance 0 chosen with one-way travel time 0:09:00 (total duration: 0 days 00:38:00)
Deploying ambulance 0 at time 2014-01-01 08:01:00
Delay on this case: 0 days 00:00:00
Busy ambulances:  [0]
Pending cases:  []

Current Time: 2014-01-01 08:33:00


In [7]:
sleep(sleeptime)
proc = processes.pop()
print(proc.name)
proc.start()
joining.append(proc)

ambs 16 bases 16
Default init_bases(): Kmeans init bases


In [8]:
# sleep(sleeptime)
# proc = processes.pop()
# print(proc.name)
# proc.start()
# joining.append(proc)

In [ ]:
for proc in joining:
    proc.join()
    print("Joined: ", proc.name)


In [ ]:
print("Done!")